In [8]:
from pathlib import Path
import sqlite3
from collections import namedtuple

import numpy as np
import sklearn

In [9]:
Article = namedtuple('Article', ['article_id', 'source_id', 'headline', 'excerpt', 'full_text', 'image_url', 'article_url'])

path_data = Path('sql')
path_data.mkdir(exist_ok=True)
db_file = path_data / 'db.sqlite'
with sqlite3.connect(str(db_file)) as con:
    cur = con.cursor()
    cur.execute('select article_id, source_id, headline, excerpt, full_text, image_url, article_url from article')
    articles = cur.fetchall()
    articles = [Article(*a) for a in articles]
    print(articles[0])

Article(article_id=1, source_id=1, headline='Boris throws down the gauntlet to May over Brexit', excerpt="Downing Street is united behind Boris Johnson's pledge to give £350 million a week to the NHS after Brexit amid speculation he is launching a leadership bid, sources claim.", full_text='Downing Street is united behind Boris Johnson\'s pledge to give £350 million a week to the NHS amid speculation he is launching a leadership bid, sources claimed this morning.\nNumber 10 reportedly confirmed the government\'s knowledge of the foreign secretary\'s ambitions after he published a 4,000-word personal manifesto for Brexit ahead of Theresa May\'s speech in Florence next week.\nThe government source said: \'Boris\' views are well known - as you\'ll see in the PM\'s speech next week the government is united.\'\xa0\nMr Johnson threw down the gauntlet to the prime minister\xa0last night warning her against giving in to Remainers in the Cabinet.\nIn an extraordinary intervention, that will rei

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
articles_contents = [a.full_text for a in articles]
tfidf = TfidfVectorizer().fit_transform(articles_contents)

In [11]:
pairwise_similarity = tfidf * tfidf.T

In [12]:
pairwise_similarity.A[0]

array([ 1.        ,  0.24714333,  0.43496607,  0.49165901,  0.22446741,
        0.24240929,  0.33550051,  0.3942955 ,  0.25619342,  0.3483313 ,
        0.37373201,  0.38626808,  0.72640857,  0.27422131,  0.26714057,
        0.28203173,  0.28171059,  0.18075044,  0.24513223,  0.28701307,
        0.24636951,  0.22298623,  0.2583599 ,  0.34339577,  0.2392699 ,
        0.23327079,  0.19338545,  0.2130068 ,  0.23590437,  0.22370076,
        0.3734685 ,  0.3624973 ,  0.35316701,  0.40121084,  0.24204002,
        0.6306044 ,  0.33130163,  0.41899866,  0.24886893,  0.29204534,
        0.08863572,  0.28456011,  0.22999557,  0.3031019 ,  0.23376341,
        0.20554271,  0.24974698,  0.23716152,  0.2934775 ,  0.3496312 ,
        0.39105806,  0.35136484,  0.24553672,  0.        ,  0.45512535,
        0.21852859,  0.31743217,  0.32268081,  0.16217817,  0.31255803,
        0.24813002,  0.21339913,  0.25834994,  0.23073153,  0.31359992,
        0.11366918,  0.31448467,  0.35293457,  0.32606735,  0.34

In [13]:
print(pairwise_similarity.A[0][12])
print(pairwise_similarity.A[0][35])
print(pairwise_similarity.A[0][36])
print(pairwise_similarity.A[12][0])
print(pairwise_similarity.A[12][35])
print(pairwise_similarity.A[12][36])
print(pairwise_similarity.A[35][0])
print(pairwise_similarity.A[35][12])
print(pairwise_similarity.A[35][36])

0.726408570948
0.630604403418
0.331301634702
0.726408570948
0.737309334754
0.374329858313
0.630604403418
0.737309334754
0.336901735621


In [14]:
ensure_pairs_exist_sql = """
INSERT INTO similarities (article_id_1, article_id_2, permid, sklearn)
  SELECT
    a1.article_id,
    a2.article_id,
    0,
    0
  FROM article a1
    INNER JOIN article a2
      ON a1.article_id > a2.article_id
  WHERE NOT EXISTS(
      SELECT *
      FROM similarities s
      WHERE s.article_id_1 == a1.article_id
            AND s.article_id_2 == a2.article_id
  );
"""

def clear_table(cur):
    cur.execute('DELETE FROM similarities')

def update_db(cur, id_1, id_2, value):
    cur.execute('UPDATE similarities SET sklearn = ? WHERE article_id_1 = ? AND article_id_2 = ?', 
                (value, id_1, id_2))

with sqlite3.connect(str(db_file)) as con:
    cur = con.cursor()
    
    clear_table(cur)
    con.commit()
        
    cur.execute(ensure_pairs_exist_sql)
    con.commit()
    
    [update_db(cur, id_1 + 1, id_2 + 1, value) 
     for (id_1, id_2), value in np.ndenumerate(pairwise_similarity.A)
     if id_1 > id_2]
    con.commit()